# 1

Реализуйте упрощенную версию byte pair encoding (без предварительного разбивания на слова, которое есть в посте о BPE на медиуме - https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10). 

Алгоритм должен работать так:
строки с текстом разбиваются на отдельные символы и далее в цикле из N итераций: а) считаются статистики встречаемости по парам символов и б) топ-K частотных пар склеиваются в один символ

Попробуйте так токенизировать текст с разными параметрами N и K. Проанализируйте словарь уникальных слов для нескольких наборов параметров - сколько уникальных слов получилось, какой токен самый длинный?

(5 баллов)

Чтобы получить 1 бонусный балл - зафиксируйте получившийся словарь и токенизируйте с помощью него текст, который ранее не встречался в корпусе (возьмите рандомную новость из яндекс новостей например). Проанализируйте насколько хорошо токенизировался текст.

*в первом задании можно использовать любые текстовые данные (например корпус lenta.txt - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/lenta.txt.zip)

In [1]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('data/lenta.txt').read()

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return " ".join(normalized_text) # склеиваю, чтобы было без предварительного разбиения на слова


In [3]:
norm_news = normalize(news)

Изначально в vocabulary хочу добавить все символы, затем буду его обновлять найденными топ-k частотными парами

In [4]:
def vocabulary(text, n, k):
    vocab = dict()

    for char in text:
        if char in vocab:
            vocab[char] += 1
        else:
            vocab[char] = 1   
    
    pairs = dict()
    for step in range(n):
        for i in vocab:
            for j in vocab:
                if i != ' ' and j != ' ':
                    temp_pair = i + j
                    if temp_pair not in pairs:
                        pairs[temp_pair] = text.count(temp_pair)
        pairs = {k: v for k, v in sorted(pairs.items(), key=lambda item: item[1], reverse=True)} # https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
    
        count = 0    
        for pair in pairs:
            if count < k:
                if pair not in vocab:
                    vocab[pair] = pairs[pair]
                    count += 1
            else:
                break  
            
    max_len = 0
    longest_token = ''
    for i in vocab:
        if len(i) > max_len:
            max_len = len(i)
            longest_token = i
    print(f"Количество уникальных символов и частотных пар символов в корпусе (с учетом n={n} и k={k}): {len(vocab)}")
    print(f"Самый длинный токен: {longest_token}")
    return vocab

In [10]:
vocabulary_1 = vocabulary(norm_news, 5, 20)

Количество уникальных символов и частотных пар символов в корпусе (с учетом n=5 и k=20): 190
Самый длинный токен: ени


In [11]:
vocabulary_1

{'б': 156501,
 'о': 1013087,
 'и': 747903,
 ' ': 1505788,
 'у': 215072,
 'с': 540120,
 'п': 286508,
 'ц': 60338,
 'к': 307168,
 'н': 639161,
 'а': 748927,
 'д': 288286,
 'р': 534412,
 'е': 798410,
 'з': 145028,
 'ч': 118051,
 'л': 376818,
 'ь': 119048,
 'т': 576976,
 'м': 280334,
 'г': 155743,
 'в': 464009,
 'я': 164508,
 'ш': 53938,
 'й': 118139,
 'ю': 55058,
 'ж': 75632,
 'ы': 163989,
 '1': 15991,
 '4': 4947,
 'х': 83278,
 'щ': 38946,
 '«': 52,
 '»': 52,
 '6': 4439,
 '9': 9227,
 '2': 11805,
 '-': 18583,
 'э': 26839,
 'ф': 36994,
 '—': 57,
 'd': 3735,
 'a': 9891,
 's': 9363,
 'i': 8448,
 't': 8397,
 'n': 8611,
 'e': 13082,
 'r': 9296,
 'o': 9557,
 'f': 2028,
 'ъ': 4707,
 '7': 4085,
 '5': 7023,
 '3': 6146,
 '№': 100,
 '…': 6,
 '–': 47,
 'l': 4872,
 'y': 1676,
 'm': 4396,
 '8': 4120,
 '0': 18494,
 'g': 1928,
 'z': 317,
 'u': 3653,
 '.': 452,
 ',': 2434,
 'b': 3071,
 'k': 1706,
 'w': 2737,
 'x': 699,
 'p': 3165,
 'h': 1926,
 'v': 1140,
 'j': 298,
 'ё': 18,
 'c': 6077,
 '/': 67,
 'q': 150

In [15]:
import copy

In [18]:
def bpe(text, vocabulary):
    
    flag = True
    old_text = list(text)     
    
    while flag:    
        new_text = list()   
        flag = False        
        n = len(old_text)        
        i = 0
        j = 1
        while j < n:
            pair = old_text[i] + old_text[j]
            if pair in vocabulary:
                new_text.append(pair)
                i += 2
                j += 2
                flag = True
            else:
                new_text.append(old_text[i])
                i = j
                j += 1
        if i == n - 1:
            new_text.append(old_text[i])
            
        old_text = copy.deepcopy(new_text)
    return new_text 

In [19]:
norm_news_bpe = bpe(norm_news, vocabulary_1)

In [28]:
temp = "".join(norm_news_bpe)

In [29]:
len(norm_news) == len(temp) # проверила, что не забыла никакой символ в bpe

True

In [30]:
for n in [5, 10, 20]:
    for k in [5, 10, 20]:
        temp_vocab = vocabulary(norm_news, n, k)
        norm_news_bpe = bpe(norm_news[:200], temp_vocab)
        print(norm_news_bpe[:200])

Количество уникальных символов и частотных пар символов в корпусе (с учетом n=5 и k=5): 115
Самый длинный токен: ст
['б', 'о', 'и', ' ', 'у', ' ', 'с', 'о', 'по', 'ц', 'к', 'и', 'на', ' ', 'и', ' ', 'д', 'р', 'у', 'с', 'к', 'ен', 'и', 'к', ' ', 'з', 'а', 'ко', 'н', 'ч', 'и', 'ли', 'с', 'ь', ' ', 'о', 'т', 'ст', 'у', 'п', 'л', 'ен', 'и', 'е', 'м', ' ', 'г', 'ер', 'м', 'ан', 'ц', 'е', 'в', ' ', 'не', 'пр', 'и', 'я', 'те', 'л', 'ь', ' ', 'пр', 'и', 'б', 'ли', 'з', 'и', 'в', 'ш', 'и', 'с', 'ь', ' ', 'с', ' ', 'с', 'е', 'в', 'ер', 'а', ' ', 'к', ' ', 'ос', 'ов', 'ц', 'у', ' ', 'на', 'ч', 'а', 'л', ' ', 'а', 'р', 'т', 'и', 'л', 'л', 'ер', 'и', 'й', 'с', 'к', 'у', 'ю', ' ', 'б', 'ор', 'ь', 'б', 'у', ' ', 'с', ' ', 'к', 'ре', 'по', 'ст', 'ь', 'ю', ' ', 'в', ' ', 'а', 'р', 'т', 'и', 'л', 'л', 'ер', 'и', 'й', 'с', 'ко', 'м', ' ', 'б', 'о', 'ю', ' ', 'пр', 'и', 'ни', 'м', 'а', 'ю', 'т', ' ', 'у', 'ч', 'а', 'ст', 'и', 'е', ' ', 'т', 'я', 'ж', 'е', 'л', 'ы', 'е', ' ', 'к', 'а', 'ли', 'б', 'р', 'ы']

Можно заметить, что с увеличением числа итераций n и с добавлением большего количества топ-k пар словарь увеличивается и в нём появляются более длинные токены. Самый длинный токен, который я увидела "росси". Можно заметить, что в результате токенизации начинаются склеиваться не только по 2 символа, но и по 4 ("тель" и "лени" например)

In [31]:
# На доп. балл

yandex_news = """В России начали разработку программы вживления чипов в мозг человека
Правительство России разрабатывает федеральную программу «Мозг, здоровье, интеллект, инновации на 2021—2029 годы» по развитию технологий нейроинтерфейсов."""

norm_yandex_news = normalize(yandex_news)

In [32]:
norm_yandex_news_bpe = bpe(norm_yandex_news, temp_vocab)
print(norm_yandex_news_bpe)

['в', ' ', 'росс', 'ии', ' ', 'на', 'ча', 'ли', ' ', 'раз', 'ра', 'бо', 'т', 'ку', ' ', 'пр', 'ог', 'ра', 'м', 'м', 'ы', ' ', 'в', 'жи', 'влен', 'ия', ' ', 'чи', 'по', 'в', ' ', 'в', ' ', 'мо', 'з', 'г', ' ', 'че', 'ло', 'ве', 'ка', ' ', 'прав', 'ител', 'ьс', 'тво', ' ', 'росс', 'ии', ' ', 'раз', 'ра', 'ба', 'ты', 'ва', 'ет', ' ', 'ф', 'ед', 'ер', 'альн', 'ую', ' ', 'пр', 'ог', 'ра', 'м', 'му', ' ', '«', ' ', 'мо', 'з', 'г', ' ', 'з', 'до', 'ров', 'ь', 'е', ' ', 'ин', 'те', 'лл', 'ек', 'т', ' ', 'ин', 'но', 'ва', 'ции', ' ', 'на', ' ', '2', '0', '2', '1', '—', '2', '0', '2', '9', ' ', 'год', 'ы', ' ', '»', ' ', 'по', ' ', 'ра', 'зв', 'ит', 'ию', ' ', 'те', 'х', 'но', 'ло', 'ги', 'й', ' ', 'не', 'й', 'ро', 'ин', 'тер', 'ф', 'ей', 'сов']


Результат получился похожим на то, что произошло с текстом из корпуса: чаще всего токенизируется по два символа, но есть и больше "влен", "росс", "ител", "прав" и т.д.

# 2

Обучите токенизатор из tokenizers на текстовом корпусе. Рассчитайте статистики для idf по корпусу, используя обученный словарь (разбейте корпус на "документы" по новым строкам, каждый "документ" токенизируйте, для каждого слова посчитайте, в скольких документах оно встречается и рассчитайте idf разделив общее количество документов на это число, возьмите логарифм от полученного числа). 
Векторизуйте текст (в мешок слов) аналогично TfidfVectorizer, используя токенизатор и idf статистики (инициализируйте*** пустую матрицу размером (N документов, K слов в словаре) и в цикле по всем документам постепенно заполните ее - токенизируйте документ, рассчитайте TF каждого слова (количество вхождений в документе поделить на общее количество слов в документе), умножьте TF на IDF и, используя индексы слов в словаре, запишите получившееся значение в матрицу)

Формулу для TFIDF можете уточнить тут -  https://ru.wikipedia.org/wiki/TF-IDF

***Чтобы инициализировать разреженную матрицу используйте scipy:***
from scipy.sparse import lil_matrix
X = lil_matrix(N, K)

Обучите классификатор на полученных векторах и оцените на кросс-валидации. 


In [33]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

In [273]:
data = pd.read_csv('dataset_ok.csv')[:30000]

In [274]:
data

,text,label
0,"наебалово века, для долбаёбов\n",INSULT
1,вся дума в таком же положении😁\n,NORMAL
2,а в каком месте массовое столкновение? шрайбик...,NORMAL
3,"значит ли это, что контроль за вывозом крупног...",NORMAL
4,вам не нужен щеночек? очень хорошие 🐶🥰\n,NORMAL
...,...,...
29995,долбоебы которое пишут эту хрень\n,INSULT
29996,"а я помню, как мы с подружками лазили белым дн...",NORMAL
29997,так поднемиесь. если есть кому. всю росию зане...,INSULT
29998,привет если поймёшь\n,NORMAL


In [36]:
!pip install tokenizers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [275]:
from tokenizers import CharBPETokenizer, Tokenizer

In [276]:
data['text'].to_csv('corpus.txt', index=None)

In [277]:
tok_sub = CharBPETokenizer()
tok_sub.train('corpus.txt', vocab_size=2000, min_frequency=10,)

In [278]:
tok_sub.encode(data.loc[0, 'text']).tokens

['на',
 'еба',
 'ло',
 'во</w>',
 'ве',
 'ка</w>',
 ',</w>',
 'для</w>',
 'дол',
 'ба',
 'ё',
 'бо',
 'в</w>']

In [279]:
data['tok_text'] = data['text'].apply(lambda x: tok_sub.encode(x).tokens)

In [280]:
temp_vocab = list(tok_sub.get_vocab().keys())

In [281]:
import math

In [282]:
def calculate_idf(terms, docs):
    idfs = list()
    for term in terms:
        count_docs_with_term = sum([1 for doc in docs if term in doc])
        if count_docs_with_term == 0:
            idf = 0
        else:
            idf = math.log(len(docs) / count_docs_with_term)
        idfs.append(idf)
    return idfs    

def calculate_tf(terms, doc):   
    tfs = list()    
    for term in terms:
        count_all = 0
        count_term = 0
        for word in doc:
            if word == term:
                count_term += 1
            count_all += 1
        tf = count_term / count_all
        tfs.append(tf)
    return tfs

In [283]:
from scipy.sparse import lil_matrix

In [285]:
N = len(data['tok_text'])
K = len(temp_vocab)
print(N, K)

30000 2000


In [286]:
%%time
tf_idf = list()
idfs = calculate_idf(temp_vocab, data['tok_text'])
for i in range(N):    
    tfs = calculate_tf(temp_vocab, data.loc[i, 'tok_text'])
    tf_idf.append([idfs[j] * tfs[j] for j in range(K)])
X = lil_matrix(tf_idf)

CPU times: user 2min 8s, sys: 2.52 s, total: 2min 10s
Wall time: 2min 10s


In [287]:
from sklearn.model_selection import cross_val_score

In [288]:
y = np.array(data.label)


In [289]:
clf = SGDClassifier(loss="log", max_iter=50)
clf.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=50,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [290]:
cross_val_score(clf, X, y, scoring="f1_micro")

array([0.88166667, 0.88166667, 0.8825    , 0.8805    , 0.882     ])

In [291]:
cross_val_score(clf, X, y, scoring="f1_micro")

array([0.88133333, 0.88183333, 0.88283333, 0.8805    , 0.88166667])